1. We use the read_cvs function of pandas.

In [1]:
import pandas as pd

file = pd.read_csv("periodic_table.csv")

2. By assuming our file (the periodic table) has two columns, we can assign each element symbol to its atomic mass, which is analogous to having a key that corresponds to a value.
   zip pairs up items from two lists (here, the lists are elements and atomic masses), which is why it comes handy in this function. 
   
   We tried to do it with "for element, atomic_number in periodic_table:" first, but that did not work. This is due to the fact that we actually have to iterate over 2 columns and not 1 column and 1 row. Therefore, we have to use zip.

In [9]:
element = file['Symbol']
atomic_mass = file['AtomicMass']
periodic_table_dictionary = dict(zip((element), atomic_mass))

3. In order to calculate the molecular weight, we used an iteration through the chemical formula as a string, which would allow us to detect an element and the number of atoms of that element. 

In [ ]:
def calculate_molecular_mass(formula: str) -> float:
    #create an empty string to store the element and count
    element = ''
    count = ''
    total_mass = 0

    i = 0
    while i < len(formula):
        character = formula[i]

        if character.isupper():
            if element:
                atomic_mass = periodic_table_dictionary[element]
                multiplier = int(count) if count else 1
                total_mass += atomic_mass * multiplier
                count = ''
            #start a new element
            element = character

        elif character.islower():
            element += character

        elif character.isdigit():
            count += character

        i += 1

    #last element in the formula
    if element:
        atomic_mass = periodic_table_dictionary[element]
        multiplier = int(count) if count else 1
        total_mass += atomic_mass * multiplier

    return total_mass

print(calculate_molecular_mass("H2O"))       
print(calculate_molecular_mass("C6H12O6"))   

18.015
180.156


4. If we now have parentheses, we have to implement recursive programming in order to get the correct molecular mass. 

In [11]:
def extended_calculate_molecular_mass(formula: str) -> float:
    #create an empty string to store the element and count
    element = ''
    count = ''
    total_mass = 0

    i = 0
    while i < len(formula):
        character = formula[i]

        if character.isupper():
            if element:
                atomic_mass = periodic_table_dictionary[element]
                multiplier = int(count) if count else 1
                total_mass += atomic_mass * multiplier
                count = ''
            #start a new element
            element = character

        elif character.islower():
            element += character

        elif character.isdigit():
            count += character
        elif character == '(':
            j = i + 1
            while formula[j] != ')':
                j += 1
            subformula = formula[i+1:j]
            sub_mass = calculate_molecular_mass(subformula)
            i = j + 1
            sub_count = ''
            while i < len(formula) and formula[i].isdigit():
                sub_count += formula[i]
                i += 1
            multiplier = int(sub_count) if sub_count else 1
            total_mass += sub_mass * multiplier
            continue

        i += 1

    #last element in the formula
    if element:
        atomic_mass = periodic_table_dictionary[element]
        multiplier = int(count) if count else 1
        total_mass += atomic_mass * multiplier

    return total_mass

print(calculate_molecular_mass("Ca(OH)2"))       

58.095


5. For coordinated water molecules, we can now add the mass of the respective water molecules that are present in the coordination complex. 

In [12]:
def coordinated_water_molecular_mass(formula):
    i = 0
    if formula[i] != '.':
        i += 1
        main_formula = formula[:i]
        hydration_part = formula[i+1:]
        hydration_count = ''
        j = 0
    while j < len(hydration_part) and hydration_part[j].isdigit():
        hydration_count += hydration_part[j]
        j += 1
    hydration_count = int(hydration_count) if hydration_count else 1
    main_mass = extended_calculate_molecular_mass(main_formula)
    water_mass = periodic_table_dictionary['H'] * 2 + periodic_table_dictionary['O']
    total_mass = main_mass + (water_mass * hydration_count)
    return total_mass

print(coordinated_water_molecular_mass("CuSO4.5H2O"))  

30.026
